In [1]:
import sys
sys.path.append("/data1/andrew/meng/mixehr/meng/VAE-EHR/src")
from importlib import reload

import numpy as np 
import pandas as pd
from tqdm import tqdm

import torch
from torch import nn, optim

In [2]:
import vae
reload(vae)

from vae import VAE, VAETrainer

In [3]:
patient_icd_file = "/data1/andrew/meng/mixehr/data/Mimic/andrew_outputs/PATIENT_ICD_BINARY.csv"
patient_icd_df = pd.read_csv(patient_icd_file, sep=' ')

patient_icd_data = patient_icd_df.drop('SUBJECT_ID', axis=1)

data = torch.tensor(patient_icd_data.values).float()
print(data.shape)

torch.Size([46520, 6984])


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print((device))

print("Feature_dim: {}".format(data.shape[1]))
model = VAE(
    feature_dim = data.shape[1], 
    encoder_dim = [(250, 500), (500, 250), (250, 100)],
    latent_dim = 10,
    decoder_dim = [(50, 100)]
)

optimizer = optim.Adam(model.parameters(), lr=0.001)
print(type(optimizer))

cuda
Feature_dim: 6984
<class 'torch.optim.adam.Adam'>


In [5]:
date="200108"
experiment_name = "{}_patient_clusters_linear_architecture".format(date)
trainer = VAETrainer(
    model=model, 
    device=device, 
    optimizer=optimizer,
    experiment_name=experiment_name
)

In [6]:
trainer.train(
    data=data, 
    epochs=20,
    batch_size=40,
    save_model_interval=5,
    clip_gradients=True,
    grad_norm_limit=5
)

  0%|          | 0/1163 [00:00<?, ?it/s]/data1/andrew/meng/mixehr/meng/VAE-EHR/src/vae.py:164: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  clip_grad_norm(self.model.parameters(), 5)
  1%|          | 11/1163 [00:00<00:11, 103.16it/s]

====> Epoch: 1 Average loss: 384910635210577852472754176.0000


  1%|          | 6/1163 [00:00<00:21, 54.76it/s]

====> Epoch: 2 Average loss: 2152.0348


 97%|█████████▋| 1124/1163 [00:14<00:00, 95.48it/s]

KeyboardInterrupt: 

In [ ]:
#model.load_state_dict(torch.load("VAE_epoch_450.pkl"))